In [1]:
from os import path, walk
import csv
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import math

In [7]:
# Find the most recent output file
file_path = ''
recent_file = '4-4-21-57.csv'
file_path = path.join('../out/occupation', recent_file)

# (
# Replacement Policy
# Access Pattern
# Associativity
# Touches
# )
rows: [(str, str, int, int)] = []
data: dict[str, list[(str, int, int)]] = {}

with open(file_path, newline='') as csv_file:
    reader = csv.reader(csv_file, quotechar='|')
    first_iter = True
    for row in reader:
        if first_iter:
            first_iter = False
            continue
        data_row = (row[1].strip(), int(row[2].strip()), int(row[3].strip()))
        if row[0] not in data:
            data[row[0]] = []
        data[row[0]].append(data_row)
        rows.append((row[0], *data_row))
print(data)
print(rows)

{'SIEVE': [('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequential', 16, 100001), ('sequent

In [9]:
repls: set[str] = set()
patterns: set[str] = set()
assocs: set[int] = set()
for row in rows:
    repls.add(row[0])
    patterns.add(row[1])
    assocs.add(row[2])
print(repls)
print(patterns)
print(assocs)

{'3Tree-fre', '3Tree-llh', 'TreePLRU', '3Tree-rln', '3Tree-lrq', '3Tree-rlh', '3Tree-rlq', '3Tree-llq', '3Tree-rre', '3Tree-frq', '3Tree-rrn', '3Tree-flq', '3Tree-rrq', '3Tree-rfn', '3Tree-fln', '3Tree-lfh', '3Tree-lfq', '3Tree-ffh', '3Tree-ffe', '3Tree-fle', '3Tree-lre', '3Tree-ffn', 'RR', '3Tree-lfn', '3Tree-lln', '3Tree-frn', '3Tree-rrh', '3Tree-rle', '3Tree-rfq', '3Tree-flh', '3Tree-frh', '3Tree-rfe', '3Tree-rfh', '3Tree-lfe', '3Tree-lrh', '3Tree-lrn', '3Tree-lle', 'SIEVE', '3Tree-ffq'}
{'repeat-3', 'repeat-4', 'double', 'repeat-5', 'sequential'}
{16, 8, 4}


In [24]:
assoc_to_ind = {16: 0, 8: 1, 4: 2}

class Statistic:
    def __init__(self, mean, median, max, min, stdev):
        self.mean = mean
        self.median = median
        self.max = max
        self.min = min
        self.stdev = stdev

stats: list[dict[(str, str), Statistic]] = [dict(), dict(), dict()]

for repl in repls:
    for assoc in assocs:
        for pattern in patterns:
            difficulty_vals = []
            for row in rows:
                if row[0] != repl or row[1] != pattern or row[2] != assoc:
                    continue
                if row[3] > 100000:
                    continue
                difficulty_vals.append(row[3])
            diff = np.array(difficulty_vals)
            stat: Statistic
            if len(diff) == 0:
                stat = Statistic(-1, -1, -1, -1, -1)
            else:
                stat = Statistic(np.mean(diff), np.median(diff), np.max(diff), np.min(diff), np.std(diff))

            stats[assoc_to_ind[assoc]][(repl, pattern)] = stat
print(stats[0])


{('3Tree-fre', 'repeat-3'): <__main__.Statistic object at 0x1164b2990>, ('3Tree-fre', 'repeat-4'): <__main__.Statistic object at 0x1163c06d0>, ('3Tree-fre', 'double'): <__main__.Statistic object at 0x1163bae10>, ('3Tree-fre', 'repeat-5'): <__main__.Statistic object at 0x11688aad0>, ('3Tree-fre', 'sequential'): <__main__.Statistic object at 0x11688a510>, ('3Tree-llh', 'repeat-3'): <__main__.Statistic object at 0x1168c9090>, ('3Tree-llh', 'repeat-4'): <__main__.Statistic object at 0x1168c9350>, ('3Tree-llh', 'double'): <__main__.Statistic object at 0x1168c94d0>, ('3Tree-llh', 'repeat-5'): <__main__.Statistic object at 0x1168c9ad0>, ('3Tree-llh', 'sequential'): <__main__.Statistic object at 0x1168c9b50>, ('TreePLRU', 'repeat-3'): <__main__.Statistic object at 0x1168c9dd0>, ('TreePLRU', 'repeat-4'): <__main__.Statistic object at 0x1168ca2d0>, ('TreePLRU', 'double'): <__main__.Statistic object at 0x1168ca350>, ('TreePLRU', 'repeat-5'): <__main__.Statistic object at 0x1168ca1d0>, ('TreePLRU'

In [35]:
# Get the minimum pattern for each replacement policy
repl_mins: list[dict[str, Statistic]] = [dict(), dict(), dict()]
for assoc in assocs:
    for repl in repls:
        min_stat: Statistic = Statistic(1000000, 0, 0, 0, 0)
        for (stat_repl, stat_pat), stat in stats[assoc_to_ind[assoc]].items():
            if stat_repl != repl:
                continue
            if min_stat.mean >= stat.mean and stat.mean != -1:
                min_stat = stat
        repl_mins[assoc_to_ind[assoc]][repl] = min_stat
print(repl_mins)

[{'3Tree-fre': <__main__.Statistic object at 0x1163bae10>, '3Tree-llh': <__main__.Statistic object at 0x1168c94d0>, 'TreePLRU': <__main__.Statistic object at 0x1168ca610>, '3Tree-rln': <__main__.Statistic object at 0x1168cb010>, '3Tree-lrq': <__main__.Statistic object at 0x1168c91d0>, '3Tree-rlh': <__main__.Statistic object at 0x1168cbd10>, '3Tree-rlq': <__main__.Statistic object at 0x1168e8350>, '3Tree-llq': <__main__.Statistic object at 0x1168e8f90>, '3Tree-rre': <__main__.Statistic object at 0x1168e9710>, '3Tree-frq': <__main__.Statistic object at 0x1168e9fd0>, '3Tree-rrn': <__main__.Statistic object at 0x1168ea750>, '3Tree-flq': <__main__.Statistic object at 0x1168eaed0>, '3Tree-rrq': <__main__.Statistic object at 0x1168eb650>, '3Tree-rfn': <__main__.Statistic object at 0x1168ebd90>, '3Tree-fln': <__main__.Statistic object at 0x1168f4110>, '3Tree-lfh': <__main__.Statistic object at 0x1168f4f10>, '3Tree-lfq': <__main__.Statistic object at 0x1168f5950>, '3Tree-ffh': <__main__.Statist

In [36]:
# for assoc in assocs:
for repl, stat in repl_mins[assoc_to_ind[16]].items():
    print(repl, stat.mean)

3Tree-fre 132.03125
3Tree-llh 121.90625
TreePLRU 16.0
3Tree-rln 67.96875
3Tree-lrq 163.375
3Tree-rlh 189.0
3Tree-rlq 110.34375
3Tree-llq 79.25
3Tree-rre 149.5
3Tree-frq 148.53125
3Tree-rrn 135.4375
3Tree-flq 79.96875
3Tree-rrq 175.40625
3Tree-rfn 80.71875
3Tree-fln 44.125
3Tree-lfh 112.96875
3Tree-lfq 72.71875
3Tree-ffh 114.3125
3Tree-ffe 56.46875
3Tree-fle 59.65625
3Tree-lre 134.53125
3Tree-ffn 44.0625
RR 200.875
3Tree-lfn 44.0625
3Tree-lln 44.15625
3Tree-frn 137.28125
3Tree-rrh 254.6875
3Tree-rle 87.15625
3Tree-rfq 100.1875
3Tree-flh 124.59375
3Tree-frh 216.0
3Tree-rfe 80.09375
3Tree-rfh 179.1875
3Tree-lfe 57.125
3Tree-lrh 214.4375
3Tree-lrn 141.90625
3Tree-lle 60.3125
SIEVE 1000000
3Tree-ffq 74.84375
